从编程的角度来看，块由类（class）表示。 它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。 在定义我们自己的块时，由于自动微分（在 2.5节 中引入） 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

In [17]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0157, -0.0731, -0.0473,  0.1667,  0.1206, -0.2215,  0.2004,  0.0514,
         -0.1355,  0.3765],
        [ 0.0749, -0.0157,  0.1386,  0.1398, -0.0270, -0.2253,  0.2022,  0.2613,
         -0.0747,  0.3973]], grad_fn=<AddmmBackward0>)

# 自定义块

- 将输入数据作为其前向传播函数的参数。
- 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
- 算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
- 存储和访问前向传播计算所需的参数。
- 根据需要初始化模型参数。

In [18]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [19]:
net = MLP()
net(X)

tensor([[-0.0923, -0.0504, -0.3088, -0.1092, -0.1840,  0.2314,  0.0406, -0.0554,
         -0.1167,  0.0932],
        [-0.2483, -0.1512, -0.3820, -0.1333, -0.1515,  0.1646,  0.0206, -0.0538,
         -0.0929,  0.0543]], grad_fn=<AddmmBackward0>)

# 顺序块
- 一种将块逐个追加到列表中的函数；
- 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [20]:
class MySequentials(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [21]:
net = MySequentials(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0004, -0.0227, -0.0167, -0.0872,  0.0450,  0.0719, -0.2447,  0.1004,
         -0.0944, -0.1435],
        [ 0.0545,  0.1007, -0.1918, -0.2021,  0.0130, -0.0456, -0.1315,  0.0325,
         -0.0807, -0.0856]], grad_fn=<AddmmBackward0>)

# 在前向传播函数中执行代码
到目前为止， 我们网络中的所有操作都对网络的激活值及网络的参数起作用。 然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）。

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [23]:
net = FixedHiddenMLP()
net(X)

tensor(0.3064, grad_fn=<SumBackward0>)

In [24]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0598, grad_fn=<SumBackward0>)